In [6]:
using BenchmarkTools, LinearAlgebra

In [7]:
#### QR, i.e. UDT decomposition
function decompose_udt(A::AbstractMatrix{C}) where C<:Number
  F = qr(A, Val(true))
  p = F.p
  @views p[p] = collect(1:length(p))
  # D = abs.(real(diag(triu(R))))
  D = abs.(real(diag(F.R)))
  T = (Diagonal(1 ./ D) * F.R)[:, p]
  return Matrix(F.Q), D, T
end

function decompose_udt!(A::AbstractMatrix{C}, D) where C<:Number
  n = length(D)
  F = qr!(A, Val(true))
  R = F.R # F.R is of regular matrix type

  @views F.p[F.p] = 1:n

  @inbounds for i in 1:n
    D[i] = abs(real(R[i,i]))
  end

  # This (very!) strangely increases the runtime of measure_tdgfs! by a ton....
  # @inbounds for i in diagind(R)
  #   R[i] = sign(real(R[i]))
  # end

  lmul!(Diagonal(1 ./ D), R)

  return Matrix(F.Q), R[:, F.p] # Q, (D is modified in-place), T  # was full(F.Q) before upgrade
end

decompose_udt! (generic function with 1 method)

In [8]:
const X = rand(10,10);
U, D, T = decompose_udt(X);
tmpD = similar(D);
tmpU1 = similar(U);
tmpU2 = similar(U);

In [18]:
"""

  inv_one_plus_udt(U, D, T) -> result

Stable calculation of [1 + UDT]^(-1):

  * Use one intermediate UDT decomposition.

Faster but less accurate than the loh approach.

Consider `inv_one_plus_udt!` as an efficient (not one-to-one) replacement.
"""
function inv_one_plus_udt(U,D,T)
#   @warn "Calling somewhat inefficient and potentially inaccurate `inv_one_plus_udt`"

  m = U' / T
  m[diagind(m)] .+= D
  u,d,t = decompose_udt(m)
  u = U * u
  t = t * T
  ldiv!(m, lu!(t), Diagonal(1 ./ d))
  m * u'
end






"""

  inv_one_plus_udt!(mc, res, U, D, T) -> nothing

Stable calculation of [1 + UDT]^(-1):

  * Use one intermediate UDT decomposition.

Uses preallocated memory in `mc`. Writes the result into `res`.

Much faster (~50%) than `inv_one_plus_udt_loh!` but less accurate.
"""
function inv_one_plus_udt_preallocated(U,D,T, d, u, t)
#   @warn "Calling potentially inaccurate `inv_one_plus_udt!`"
  m = U' / T
  m[diagind(m)] .+= D

  utmp,ttmp = decompose_udt!(m, d)
  mul!(u, U, utmp)
  mul!(t, ttmp, T)
    
  ldiv!(m, lu!(t), Diagonal(1 ./ d))
    
  return m * u'
end

inv_one_plus_udt_preallocated

In [19]:
@btime inv_one_plus_udt($U, $D, $T);
@btime inv_one_plus_udt_preallocated($U, $D, $T, $tmpD, $tmpU1, $tmpU2);

  35.600 μs (41 allocations: 50.91 KiB)
  34.600 μs (33 allocations: 46.06 KiB)


In [9]:
struct inv_one_plus_udt_functor{D<:AbstractVector{<:Real}, U<:AbstractMatrix}
    tmpD::D
    tmpU1::U
    tmpU2::U
end

ErrorException: invalid redefinition of constant inv_one_plus_udt_functor

In [20]:
function (f::inv_one_plus_udt_functor)(U, D, T)
    inv_one_plus_udt_preallocated(U, D, T, f.tmpD, f.tmpU1, f.tmpU2)
end

In [21]:
f = inv_one_plus_udt_functor(tmpD, tmpU1, tmpU2);
typeof(f)

inv_one_plus_udt_functor{Array{Float64,1},Array{Float64,2}}

In [22]:
f(U, D, T)

10×10 Array{Float64,2}:
  0.66706      0.0826716   0.123307  …  -0.329718    0.149039    0.015834  
 -0.157469     0.707085    0.373679     -0.157718    0.211471    0.00370892
  0.0347695   -0.890985    1.42732       0.39504    -0.753529    0.576347  
 -0.0519695    0.782845   -0.349997     -0.362364    0.44376    -0.105318  
 -0.12954     -0.316583    0.171504      0.0927722  -0.219818    0.3127    
 -0.0697467   -0.121193   -0.41672   …   0.147586   -0.33785    -0.38575   
  0.4699      -1.03179     0.261533     -0.599686   -0.122059    0.298605  
 -0.6367       0.797497   -0.981257      1.30038    -0.0914739  -0.825313  
 -0.00577522  -0.205844   -0.388198     -0.0473768   0.567224   -0.389856  
  0.14235     -0.200328    0.385908     -0.302282   -0.0384837   0.987121  

In [23]:
@btime f($U,$D,$T);

  34.600 μs (33 allocations: 46.06 KiB)


In [24]:
function use_functor_but_reallocate(U,D,T)
    f = inv_one_plus_udt_functor(similar(D), similar(U), similar(U));
    f(U,D,T)
end

use_functor_but_reallocate (generic function with 1 method)

In [28]:
@btime use_functor_but_reallocate($U,$D,$T);

  35.301 μs (36 allocations: 47.97 KiB)


# Benchmark all

In [29]:
@btime inv_one_plus_udt($U, $D, $T);
@btime inv_one_plus_udt_preallocated($U, $D, $T, $tmpD, $tmpU1, $tmpU2);
@btime f($U,$D,$T);
@btime use_functor_but_reallocate($U,$D,$T);

  35.300 μs (41 allocations: 50.91 KiB)
  34.501 μs (33 allocations: 46.06 KiB)
  33.601 μs (33 allocations: 46.06 KiB)
  34.501 μs (36 allocations: 47.97 KiB)


In [30]:
@btime inv_one_plus_udt($U, $D, $T);
@btime inv_one_plus_udt_preallocated($U, $D, $T, $tmpD, $tmpU1, $tmpU2);
@btime f($U,$D,$T);
@btime use_functor_but_reallocate($U,$D,$T);

  34.600 μs (41 allocations: 50.91 KiB)
  34.700 μs (33 allocations: 46.06 KiB)
  35.000 μs (33 allocations: 46.06 KiB)
  33.900 μs (36 allocations: 47.97 KiB)
